In [1]:
import pandas as pd
import numpy as np
# import os
import nltk
import re
# nltk.data.path.append(r"C:\Users\liufa\nltk_data")

# from nltk.tokenize import word_tokenize
# nltk.download('brown')
import matplotlib.pyplot as plt
import collections
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


# BERT
import tensorflow as tf
import keras
from tensorflow.keras import backend as K

from tensorflow.keras.preprocessing.text import one_hot, Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Dense, Flatten, Lambda,\
    GlobalMaxPooling1D, LSTM, Bidirectional, concatenate, Embedding, multiply
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model

In [2]:
# raw_data=pd.read_csv(r'../input/commonlitreadabilityprize/train.csv')
raw_data=pd.read_csv(r'C:/Users/liufa/Desktop/Kaggle competition/train.csv')

df = raw_data.rename(columns={'excerpt':'text','target':'y'})
df.head(5)

,id,url_legal,license,text,y,standard_error
0,c12129c31,NaN,NaN,When the young people returned to the ballroom...,-0.340259,0.464009
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805
2,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676
3,dd1000b26,NaN,NaN,And outside before the palace a great garden w...,-1.054013,0.450007
4,37c1b32fb,NaN,NaN,Once upon a time there were Three Bears who li...,0.247197,0.510845


In [3]:
import string
string.punctuation

def remove_punct(text):
    text_nopunct=''.join([char for char in text if char not in string.punctuation])
    return text_nopunct

df['text_no_punct']=df['text'].apply(lambda x: remove_punct(x))

def tokenize(text):
    tokens=re.split('\W+',text)
    return tokens
df['text_tokenized']=df['text_no_punct'].apply(lambda x: tokenize(x.lower()))

stopwords=nltk.corpus.stopwords.words('english')
def remove_stopwords(tokenized_list):
    text=[word for word in tokenized_list if word not in stopwords]
    return text
df['text_no_stopwords']=df['text_tokenized'].apply(lambda x: remove_stopwords(x))

wn=nltk.WordNetLemmatizer()
def lemmatizing(tokenized_text):
    text=[wn.lemmatize(word) for word in tokenized_text]
    return text
df['text_lemmatizer']=df['text_no_stopwords'].apply(lambda x: lemmatizing(x))

df['text_clean']=df['text_lemmatizer'].apply(lambda x: ' '.join(x))
df.head()


,id,url_legal,license,text,y,standard_error,text_no_punct,text_tokenized,text_no_stopwords,text_lemmatizer,text_clean
0,c12129c31,NaN,NaN,When the young people returned to the ballroom...,-0.340259,0.464009,When the young people returned to the ballroom...,"[when, the, young, people, returned, to, the, ...","[young, people, returned, ballroom, presented,...","[young, people, returned, ballroom, presented,...",young people returned ballroom presented decid...
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805,All through dinner time Mrs Fayre was somewhat...,"[all, through, dinner, time, mrs, fayre, was, ...","[dinner, time, mrs, fayre, somewhat, silent, e...","[dinner, time, mr, fayre, somewhat, silent, ey...",dinner time mr fayre somewhat silent eye resti...
2,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676,As Roger had predicted the snow departed as qu...,"[as, roger, had, predicted, the, snow, departe...","[roger, predicted, snow, departed, quickly, ca...","[roger, predicted, snow, departed, quickly, ca...",roger predicted snow departed quickly came two...
3,dd1000b26,NaN,NaN,And outside before the palace a great garden w...,-1.054013,0.450007,And outside before the palace a great garden w...,"[and, outside, before, the, palace, a, great, ...","[outside, palace, great, garden, walled, round...","[outside, palace, great, garden, walled, round...",outside palace great garden walled round fille...
4,37c1b32fb,NaN,NaN,Once upon a time there were Three Bears who li...,0.247197,0.510845,Once upon a time there were Three Bears who li...,"[once, upon, a, time, there, were, three, bear...","[upon, time, three, bears, lived, together, ho...","[upon, time, three, bear, lived, together, hou...",upon time three bear lived together house wood...


In [4]:
def count_punct(text):
    ct=sum([1 for char in text if char in string.punctuation])
    return round(ct/(len(text)-text.count(' ')),3)

def count_capital_words(text):
    return sum(map(str.isupper,text.split()))
    
def count_sent(text):
    return len(nltk.sent_tokenize(text))

def count_stopwords(text):
    stopwords=nltk.corpus.stopwords.words('english')
    word_tokens = nltk.word_tokenize(text)
    n_stopwords = [w for w in word_tokens if w in stopwords]
    return len(n_stopwords)

def count_unique_words(text):
    return len(set(text.split()))

df['num_char']=df['text'].apply(lambda x: len(x)-x.count(' '))
df['num_words']=df['text_tokenized'].apply(lambda x: len(x))
df['num_sent']=df['text'].apply(lambda x: count_sent(x))
df['avg_word_len'] = df['num_char']/df['num_words']
df['avg_sent_len'] = df['num_char']/df['num_sent']
df['num_cap_words'] = df['text'].apply(lambda x: count_capital_words(x))
df['punct_ratio']=df['text'].apply(lambda x:count_punct(x))
df['stop_words_ratio'] = df['text'].apply(lambda x: count_stopwords(x))/df['num_words']
df['unique_words_ratio'] = df['text'].apply(lambda x: count_unique_words(x))/df['num_words']

df.head()

,id,url_legal,license,text,y,standard_error,text_no_punct,text_tokenized,text_no_stopwords,text_lemmatizer,text_clean,num_char,num_words,num_sent,avg_word_len,avg_sent_len,num_cap_words,punct_ratio,stop_words_ratio,unique_words_ratio
0,c12129c31,NaN,NaN,When the young people returned to the ballroom...,-0.340259,0.464009,When the young people returned to the ballroom...,"[when, the, young, people, returned, to, the, ...","[young, people, returned, ballroom, presented,...","[young, people, returned, ballroom, presented,...",young people returned ballroom presented decid...,819,179,11,4.575419,74.454545,0,0.033,0.452514,0.636872
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805,All through dinner time Mrs Fayre was somewhat...,"[all, through, dinner, time, mrs, fayre, was, ...","[dinner, time, mrs, fayre, somewhat, silent, e...","[dinner, time, mr, fayre, somewhat, silent, ey...",dinner time mr fayre somewhat silent eye resti...,774,169,15,4.579882,51.600000,6,0.072,0.396450,0.751479
2,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676,As Roger had predicted the snow departed as qu...,"[as, roger, had, predicted, the, snow, departe...","[roger, predicted, snow, departed, quickly, ca...","[roger, predicted, snow, departed, quickly, ca...",roger predicted snow departed quickly came two...,747,166,11,4.500000,67.909091,2,0.063,0.451807,0.771084
3,dd1000b26,NaN,NaN,And outside before the palace a great garden w...,-1.054013,0.450007,And outside before the palace a great garden w...,"[and, outside, before, the, palace, a, great, ...","[outside, palace, great, garden, walled, round...","[outside, palace, great, garden, walled, round...",outside palace great garden walled round fille...,747,164,5,4.554878,149.400000,0,0.044,0.414634,0.719512
4,37c1b32fb,NaN,NaN,Once upon a time there were Three Bears who li...,0.247197,0.510845,Once upon a time there were Three Bears who li...,"[once, upon, a, time, there, were, three, bear...","[upon, time, three, bears, lived, together, ho...","[upon, time, three, bear, lived, together, hou...",upon time three bear lived together house wood...,577,147,5,3.925170,115.400000,0,0.055,0.496599,0.346939


In [5]:
# Topic modeling

In [9]:
# Train test split
cols = [c for c in df.columns if c not in ['id',
                                           'url_legal',
                                           'license',
                                           'standard_error',
                                           'text',
                                           'text_clean',
                                           'text_no_punct',
                                           'text_tokenized',
                                           'text_no_stopwords',
                                           'text_lemmatizer',
                                           'y']]

train, test = train_test_split(df, test_size=0.2, random_state=42)

X_train_num = train[cols].to_numpy()
y_train = train[['y']].values


# X_test_num = X_test[[c for c in X.columns if c != 'text']].to_numpy()
# y_test = y_test.values


# standard scaler
X_scaler = StandardScaler().fit(X_train_num)
X_train_scaled = X_scaler.transform(X_train_num)

# TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA, TruncatedSVD

tfidf = TfidfVectorizer().fit(train['text_clean'])
X_vectorized=tfidf.transform(train['text_clean']).toarray()

tfidf_scaler = StandardScaler().fit(X_vectorized)
X_tfidf_scaled = tfidf_scaler.transform(X_vectorized)

comp = 600
pca = PCA(n_components=comp, random_state=0).fit(X_tfidf_scaled)
X_tfidf_pca = pca.transform(X_tfidf_scaled)

# X_train_scaled = np.concatenate((X_train_scaled, X_tfidf_pca), axis=1)
# X_train_scaled

In [10]:
y_train

array([[-1.51834981],
       [-0.54880659],
       [-0.19326203],
       ...,
       [-1.58438395],
       [-2.03468808],
       [-0.6052157 ]])

In [11]:
# help functions

# to define 'rmse' as loss instead of 'mse'
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true))) 

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', 
    factor = 0.5 , 
    patience=1, 
    mode='min', 
    verbose=1)

early_stop = keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=2,
    verbose=1,
    restore_best_weights=True)

In [29]:
# numerical features

num_feat = X_train_scaled.shape[1]
tfidf_feat = X_tfidf_pca.shape[1]

numeric_input = tf.keras.Input(shape=(num_feat,),dtype='float32')
tfidf_input = tf.keras.Input(shape=(tfidf_feat,),dtype='float32')

num_layer = Dense(10, activation='relu')(numeric_input)

tfidf_layer = Dense(300, activation='relu')(tfidf_input)
tfidf_layer = Dropout(0.2)(tfidf_layer)

layer = concatenate([num_layer, num_layer])

layer = Dense(50, activation='relu')(layer)
out = Dense(1, activation='linear')(layer)


model = tf.keras.Model(inputs=[numeric_input,tfidf_input], outputs=out)

model.compile(optimizer=Adam(0.001), loss=rmse) 
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 9)]          0                                            
__________________________________________________________________________________________________
dense_6 (Dense)                 (None, 10)           100         input_5[0][0]                    
__________________________________________________________________________________________________
concatenate_2 (Concatenate)     (None, 20)           0           dense_6[0][0]                    
                                                                 dense_6[0][0]                    
__________________________________________________________________________________________________
dense_8 (Dense)                 (None, 50)           1050        concatenate_2[0][0]        

In [30]:
history = model.fit([X_train_scaled,X_tfidf_pca], y_train,
                    epochs=15, verbose=1, batch_size=128, validation_split=0.3, callbacks=[reduce_lr,early_stop])

# X_test_scaled = X_scaler.transform(X_test_num)
# score = model.evaluate(X_test_scaled, y_test, verbose=1)
# print("Test Scores:", score)

del model
K.clear_session()

Epoch 1/15
13/13 [==============================] - 1s 10ms/step - loss: 1.2158 - val_loss: 1.0887
Epoch 2/15
13/13 [==============================] - 0s 4ms/step - loss: 1.0667 - val_loss: 0.9627
Epoch 3/15
13/13 [==============================] - 0s 4ms/step - loss: 0.9719 - val_loss: 0.9004
Epoch 4/15
13/13 [==============================] - 0s 4ms/step - loss: 0.9317 - val_loss: 0.8818
Epoch 5/15
13/13 [==============================] - 0s 4ms/step - loss: 0.9146 - val_loss: 0.8688
Epoch 6/15
13/13 [==============================] - 0s 4ms/step - loss: 0.9017 - val_loss: 0.8559
Epoch 7/15
13/13 [==============================] - 0s 4ms/step - loss: 0.8904 - val_loss: 0.8498
Epoch 8/15
13/13 [==============================] - 0s 4ms/step - loss: 0.8814 - val_loss: 0.8417
Epoch 9/15
13/13 [==============================] - 0s 4ms/step - loss: 0.8744 - val_loss: 0.8364
Epoch 10/15
13/13 [==============================] - 0s 4ms/step - loss: 0.8674 - val_loss: 0.8300
Epoch 11/15
13/13 

In [ ]:
from kerastuner.tuners import BayesianOptimization
def build_model(hp):
    input_ids = tf.keras.Input(shape=(max_len, ),dtype='int32')
    attention_mask = tf.keras.Input(shape=(max_len, ),dtype='int32')

    feat_num = X_train_num.shape[1]
    numeric_input = tf.keras.Input(shape=(feat_num,),dtype='float32')

    # change huggingface model if needed
    bert_layer = TFAutoModel.from_pretrained(model_path)(input_ids, attention_mask)[0]

    layer = Flatten()(bert_layer)
    layer = Dense(769, activation='relu')(layer)
    layer = Concatenate(axis=1)([layer, numeric_input])
    layer = Dense(hp.Int('dense_units1',min_value=40,max_value=300,step=10), activation='relu')(layer)
    layer = Dropout(0.2)(layer)
    out = Dense(1, activation='linear')(layer)

    model = tf.keras.Model(inputs=[input_ids , attention_mask, numeric_input], outputs=out)

#     model.compile(optimizer=Adam(2e-5), loss='mse', metrics=['mean_squared_error']) 
    model.compile(optimizer=Adam(2e-5), loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError()]) 
    return model

In [ ]:
import os

bayesian_opt_tuner = BayesianOptimization(
    build_model,
    objective='val_mean_squared_error',
    max_trials=10,
    executions_per_trial=1,
    directory=os.path.normpath(r'C:\Users\liufa\keras_tuning'),
    project_name='kerastuner_bayesian_poc',
    overwrite=True)

bayesian_opt_tuner.search([X_train_tokenized['input_ids'],X_train_tokenized['attention_mask'],X_train_num],\
                           y_train, epochs=15, batch_size=2, callbacks=[early_stop], validation_split=0.3, verbose=1)
     #validation_data=(X_test, y_test)
#     callbacks=[early_stop], 

bayes_opt_model_best_model = bayesian_opt_tuner.get_best_models(num_models=2)

In [ ]:
bayes_opt_model_best_model[0].summary()
best_hp = bayesian_opt_tuner.get_best_hyperparameters(1)[0]
print(best_hp.values)

In [ ]:
################################ Submission ##########################################

df_test = pd.read_csv(r'../input/commonlitreadabilityprize/test.csv')

df_test = df_test.rename(columns={'excerpt':'text','target':'y'})
df_test['text_no_punct']=df_test['text'].apply(lambda x: remove_punct(x))
df_test['text_tokenized']=df_test['text_no_punct'].apply(lambda x: tokenize(x.lower()))
stopwords=nltk.corpus.stopwords.words('english')
df_test['text_no_stopwords']=df_test['text_tokenized'].apply(lambda x: remove_stopwords(x))
wn=nltk.WordNetLemmatizer()
df_test['text_lemmatizer']=df_test['text_no_stopwords'].apply(lambda x: lemmatizing(x))
df_test['text_clean']=df_test['text_lemmatizer'].apply(lambda x: ' '.join(x))
df_test['sentence_len']=df_test['text'].apply(lambda x: len(x)-x.count(' '))
df_test['word_len']=df_test['text_tokenized'].apply(lambda x: len(x))
df_test['punct_count']=df_test['text'].apply(lambda x:count_punct(x))




In [ ]:
df_test['target'] = ridge.predict(X_test_ridge)
output = df_test[['id','target']]
output.to_csv('./submission.csv', index=False)